In [1]:
## for getting a known time of max light.
## pandas datafram to access .xls or .csv file generated with aij

import pandas as pd
import os
from pathlib import Path
print(os.getcwd())

/data/cartermg/code


In [35]:
# gets file
pathname = Path('msmts.csv') # uses a recent time of known max light. created a .csv from an AIJ measurements file.
df = pd.read_csv(pathname, sep='\t')

# reads values, gets time of max light
# PEAK MUST EXIST IN DURATION OF OBSERVATION
idxmax = df['rel_flux_T1'].idxmax()  
T0_BJDTBD = df.loc[idxmax, 'BJD_TDB']  # corresponding BJD_TDB

P = 0.13053502226702715    # in d. period calculated in 329 for V927 Her. Needs to be edited if a different target.

print(f'Time of max light during OPO observation: {T0_BJDTBD}')

Time of max light during OPO observation: 2460761.946121


In [36]:
## for finding the target star coordinates, to be used in HJD and BJD conversion
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from astropy.coordinates import EarthLocation
import astropy.units as u

In [37]:
def simbad_coords(object_name):
    # query Simbad
    result = Simbad.query_object(object_name)
    if result is None:
        raise ValueError(f"Object '{object_name}' not found in Simbad.")
    
    ra = result['RA'][0]  # in 'hh mm ss' format
    dec = result['DEC'][0]  # in 'dd mm ss' format
    print(f'Star coordinates found: {ra}, {dec}')
    # convert to SkyCoord
    coord = SkyCoord(f"{ra} {dec}", unit=(u.hourangle, u.deg), frame='icrs')

    return coord

In [38]:
target_coords = simbad_coords('V927 Her')

Star coordinates found: 16 56 17.9985, +50 07 35.856


In [39]:
## sets observatory location.

obs_location = EarthLocation.from_geodetic(lon=-105.820417 * u.deg, lat=32.780361 * u.deg, height=2788 * u.m)
print(f"APO Earth Location Coordinates: {obs_location}")

APO Earth Location Coordinates: (-1463998.7009965, -5166633.73441486, 3435013.97233452) m


In [40]:
## finds the phase.
## includes target coordinates from simbad_coords function.
## make sure to switch to HJD if that is the time used from the .csv file.

from astropy.io import fits
from astropy.time import Time

In [42]:
def calculate_phase(fits_file, T0, period, location, target_coord, T0_system='BJD'):
    """
    parameters:
    - fits_file: path to the FITS file
    - T0: time of max light (in HJD or BJD depending on T0_system)
    - period: period of the star (days)
    - location: astropy EarthLocation object for the observatory
    - target_coord: astropy SkyCoord object for the star
    - T0_system: 'HJD' or 'BJD' depending on what T0 is

    returns:
    - phase: oscillatory phase (0, 1]
    - time_obs: whatever time object is given in header for printing. typically UT.
    - date_obs: whatever date object is given in header for printing. typically UT. 
    """

    # open FITS file and read MJD
    hdul = fits.open(fits_file)
    header = hdul[0].header
    mjd_obs = header.get('AVE_MJD') or header.get('MJD') or header.get('MJD-OBS') # check for dao data
    hdul.close()

    if mjd_obs is None:
        raise ValueError(f"No MJD keyword found in {fits_file}")

    # create a time object from MJD
    time_obs = Time(mjd_obs, format='mjd', location=location)

    # calculate light travel time correction
    ltt_bary = time_obs.light_travel_time(target_coord, kind='barycentric')
    ltt_helio = time_obs.light_travel_time(target_coord, kind='heliocentric')

    # corrected times
    bjd_tdb = time_obs.tdb + ltt_bary
    hjd_utc = time_obs.utc + ltt_helio

    # choose which corrected time to use based on T0_system
    if T0_system.upper() == 'BJD':
        obs_corrected = bjd_tdb
    elif T0_system.upper() == 'HJD':
        obs_corrected = hjd_utc
    else:
        raise ValueError("T0_system must be either 'HJD' or 'BJD'.")

    # calculate phase
    phase = ((obs_corrected.value - T0) / period) % 1

    # for returning date and time
    date_obs = header.get('UT-DATE') or header.get('DATE') or header.get('AVE_DATE')
    time_obs = header.get('TIME') or header.get('AVE_TIME') or header.get('UT')

    return phase, time_obs, date_obs

In [43]:
## gets file names in the target folder

def get_file_names(folder_path):
  """
  args:
    folder_path: path to the folder.

  returns:
    a list of strings, where each string is a file name in the folder.
    returns an empty list if the folder doesn't exist or is not a directory.
  """
  if not os.path.isdir(folder_path):
    print(f"Error: '{folder_path}' is not a valid directory.")
    return []
  try:
    file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith(".fits")]
    return file_names
  except OSError as e:
    print(f"Error accessing '{folder_path}': {e}")
    return []

In [47]:
## writes the periods to a text file in the target directory.

def write_periods(folder_path, list):
    """
    args:
        folder_path: the path to the folder
    
    returns:
        none. writes a .txt with filenames and their periods.
    """
    try: 
        outfile = os.path.join(folder_path, 'periods.txt')
        
        with open(outfile, 'w') as file:
            file.write(f'SORTED PERIODS: \n')
            for object in list:
                file.write(f'For file {object[3]}:: Phase: {object[0]}, at time {object[1]} on {object[2]} \n')

        print(f'File write complete. Ordered periods with times and filenames available in {outfile}')
    except OSError as e:
        print(f'Error writing to file: {e}')
        
    return

In [49]:
## pick a directory and then run the final function.

data_location = Path('/data/cartermg/V927Her/opticalspectra/may2019/')
filenames = get_file_names(data_location)
pf_list = [] 

for filename in filenames:
    target_file = data_location.joinpath(filename)
    phase = calculate_phase(target_file, T0_BJDTBD, P, obs_location, target_coords)
    pairs = (phase[0], phase[1], phase[2], filename)
    pf_list.append(pairs)
    print(f'Phase: {phase[0]}, at time {phase[1]} on {phase[2]}')

pf_list.sort()

write_periods(data_location, pf_list)

Phase: 0.9472885057330132, at time 10:04:49 on 2019-05-05
Phase: 0.028799723833799362, at time 10:20:08 on 2019-05-05
Phase: 0.28280723839998245, at time 10:07:25 on 2019-05-07
Phase: 0.3642417602241039, at time 10:22:44 on 2019-05-07
File write complete. Ordered periods with times and filenames available in /data/cartermg/V927Her/opticalspectra/may2019/periods.txt
